# Not Scaled Data

In [53]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)
def find_y(key, keys):
    for ind,i in enumerate(keys):
        if key in i:
            return ind

In [54]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
data = dict()
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = texts['P3:' + key1]
print(data.keys())


dict_keys([' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_'])


# The First Union

In [55]:
import numpy as np
Y = []
X = []
for key in keys:
    y = find_y(key, keys2)
    for text in data[key]:
        X.append(text)
        Y.append(y)
y = np.array(Y)
len(X)

267

In [56]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y)
y = lb.transform(y)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)
X = X.toarray()

In [61]:
import torch
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

from torch import nn
import torch.nn.functional as F

print(X_train.shape)
model = nn.Sequential()
model.add_module(name='layer1', module=nn.Linear(4085, 1000))
model.add_module(name='layer2', module=nn.ReLU())
model.add_module(name='layer3', module=nn.Linear(1000, 1000))
model.add_module(name='layer4', module=nn.ReLU())
model.add_module(name='layer5', module=nn.Linear(1000, 100))
model.add_module(name='layer6', module=nn.ReLU())
model.add_module(name='layer7', module=nn.Linear(100, 100))
model.add_module(name='layer8', module=nn.ReLU())
model.add_module(name='layer9', module=nn.Linear(100, 3))
model.add_module(name='layer10', module=nn.Softmax(dim = 1))

base_model = model

x_train = torch.tensor(X_train, dtype=torch.float32)
#y_train = torch.tensor(y_train, dtype=torch.float32)
class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
class_weight = [1/j for j in class_weight]
class_weight

(200, 4085)


[0.007042253521126761, 0.034482758620689655, 0.034482758620689655]

In [64]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

history = []
for i in range(500):
    
    ix = np.random.randint(0, len(X_train), 10)
    x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
    y_batch = torch.tensor(y_train[ix], dtype=torch.float32)
    
    # predict probabilities
    y_predicted = model(x_batch)

    # compute loss, just like before
    loss = F.binary_cross_entropy(y_predicted, y_batch)
    
    # compute gradients
    loss.backward()
    
    # Adam step
    opt.step()
    
    # clear gradients
    opt.zero_grad()
    
    history.append(loss.data.numpy())
    
    if i % 100 == 0:
        print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))
print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))

step #0 | mean loss = 0.000001
step #100 | mean loss = 0.000000
step #200 | mean loss = 0.000000
step #300 | mean loss = 0.000000
step #400 | mean loss = 0.000000
step #499 | mean loss = 0.000000


In [65]:
from sklearn.metrics import f1_score
y_pred = model(torch.tensor(X_test, dtype=torch.float32)).data.numpy()
f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro')

0.3438515968636451

# The full feature

In [66]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [67]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [68]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [69]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [70]:
import numpy as np
from sklearn.preprocessing import scale
X_1 = np.vstack([data[i].values for i in keys])
X_1 = X_1[:, 1:]
X_1.shape

(267, 243)

In [71]:
X.shape

(267, 4085)

In [72]:
X = np.hstack([X, X_1])
X.shape

(267, 4328)

In [80]:
import torch
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

from torch import nn
import torch.nn.functional as F

print(X_train.shape)
model = nn.Sequential()
model.add_module(name='layer1', module=nn.Linear(4328, 10000))
model.add_module(name='layer2', module=nn.ReLU())
model.add_module(name='drop', module=nn.Dropout())
model.add_module(name='layer3', module=nn.Linear(10000, 1000))
model.add_module(name='layer4', module=nn.ReLU())
model.add_module(name='drop', module=nn.Dropout())
model.add_module(name='layer5', module=nn.Linear(1000, 100))
model.add_module(name='layer6', module=nn.ReLU())
model.add_module(name='layer7', module=nn.Linear(100, 3))
model.add_module(name='layer10', module=nn.Softmax(dim = 1))

base_model = model

x_train = torch.tensor(X_train, dtype=torch.float32)
#y_train = torch.tensor(y_train, dtype=torch.float32)
class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
class_weight = [1/j for j in class_weight]
class_weight

(200, 4328)


[0.006944444444444444, 0.043478260869565216, 0.030303030303030304]

In [81]:
opt = torch.optim.Adam(model.parameters(), lr=1e-4)

history = []
for i in range(400):
    
    ix = np.random.randint(0, len(X_train), 10)
    x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
    y_batch = torch.tensor(y_train[ix], dtype=torch.float32)
    
    y_predicted = model(x_batch)

    loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))
    
    loss.backward()
    
    opt.step()
    
    opt.zero_grad()
    
    history.append(loss.data.numpy())
    
    if i % 100 == 0:
        print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))
print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))

step #0 | mean loss = 0.013172
step #100 | mean loss = 0.009012
step #200 | mean loss = 0.009473
step #300 | mean loss = 0.005328
step #399 | mean loss = 0.002481


In [82]:
from sklearn.metrics import f1_score
y_pred = model(torch.tensor(X_test, dtype=torch.float32)).data.numpy()
f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro')

0.24358974358974358

In [103]:
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import tqdm

class TwoInputsNet(nn.Module):
    def __init__(self, n_tokens, n_psycho_features, concat_number_of_features, hid_size=100):
        super(TwoInputsNet, self).__init__()
        self.words = []
        self.words.append(nn.Linear(in_features = n_tokens, out_features = 1000))
        self.words.append(nn.Dropout())
        self.words.append(nn.ReLU())
        self.words.append(nn.Linear(in_features = 1000, out_features = 1000))
        self.words.append(nn.Dropout())
        self.words.append(nn.ReLU())
        self.words.append(nn.Linear(in_features = 1000, out_features = hid_size))
        self.words.append(nn.ReLU())
        [self.add_module(name = "words_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.words)]



        self.psycho = []
        self.psycho.append(nn.Linear(in_features = n_psycho_features, out_features = 100)
)
        self.words.append(nn.Dropout())
        self.psycho.append(nn.ReLU())
        self.psycho.append(nn.Linear(in_features = 100, out_features = concat_number_of_features - hid_size)
)
        self.words.append(nn.Dropout())
        self.psycho.append(nn.ReLU())
        [self.add_module(name = "psycho_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.words)]

        # Example for the final layers (after the concatenation)
        self.final_denses = []
        self.final_denses.append(nn.BatchNorm1d(concat_number_of_features))
        self.final_denses.append(nn.Linear(in_features=concat_number_of_features, out_features=100))
        self.final_denses.append(nn.ReLU())
        self.final_denses.append(nn.Linear(in_features=100, out_features=3))
        self.final_denses.append(nn.Softmax(dim = 1))
        [self.add_module(name = "final_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.final_denses)]

        

    def forward(self, whole_input):
        input1, input2  = whole_input
        for layer in self.words:
            input1 = layer(input1)
        
        for layer in self.psycho:
            input2 = layer(input2)
        concatenated = torch.cat(
            [
                input1.view(input1.size(0), -1),
                input2.view(input2.size(0), -1)
            ],
            dim=1)
        out = concatenated
        for layer in self.final_denses:
            out = layer(out)
        
        return out

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = TwoInputsNet(4085, 4328-4085, 200)
class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
class_weight = [1/j for j in class_weight]
class_weight

[0.006993006993006993, 0.041666666666666664, 0.030303030303030304]

In [129]:

opt = torch.optim.Adam(model.parameters(), lr=1e-4)

history = []
for i in range(2400):
    
    ix = np.random.randint(0, len(X_train), 10) 
    x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
    y_batch = torch.tensor(y_train[ix], dtype=torch.float32)
    
    y_predicted = model((x_batch[:, :4085], x_batch[:, 4085:]))

    loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))
    
    loss.backward()
    
    opt.step()
    
    opt.zero_grad()
    
    history.append(loss.data.numpy())
    
    if i % 100 == 0:
        print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))
print("step #%i | mean loss = %.6f" % (i, np.mean(history[-10:])))

step #0 | mean loss = 0.014263
step #100 | mean loss = 0.012795
step #200 | mean loss = 0.011310
step #300 | mean loss = 0.011490
step #400 | mean loss = 0.010751
step #500 | mean loss = 0.010306
step #600 | mean loss = 0.011058
step #700 | mean loss = 0.008826
step #800 | mean loss = 0.007771
step #900 | mean loss = 0.006913
step #1000 | mean loss = 0.004849
step #1100 | mean loss = 0.003453
step #1200 | mean loss = 0.003835
step #1300 | mean loss = 0.003351
step #1400 | mean loss = 0.002351
step #1500 | mean loss = 0.002010
step #1600 | mean loss = 0.002166
step #1700 | mean loss = 0.001178
step #1800 | mean loss = 0.001394
step #1900 | mean loss = 0.001195
step #2000 | mean loss = 0.000578
step #2100 | mean loss = 0.000807
step #2200 | mean loss = 0.000931
step #2300 | mean loss = 0.000947
step #2399 | mean loss = 0.000280


In [130]:
from sklearn.metrics import f1_score
x = torch.tensor(X_test, dtype=torch.float32)
y_pred = model((x[:, :4085], x[:, 4085:])).data.numpy()
f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro')

0.3599257884972171